# Apprenez une heuristique pour le Go

Dans ce **TP noté**, vous devrez déployer des methodes d'apprentissage automatique permettant d'évaluer la qualité de plateaux de GO.

Pour cela, vous disposerez de 41563 exemples de plateau de Go, tous générés par gnugo après quelques coups contre lui même avec un niveau de difficulté de 0. Par chaque plateau, nous avons lancé 100 matchs de gnugo contre lui même, toujours avec un niveau 0, et compté le nombre de victoires de noir et de blanc depuis ce plateau.

A noter, chaque "rollout" (un rollout et un déroulement possible du match depuis le plateau de référence) correspond à des mouvements choisis aléatoirement parmis les 10 meilleurs mouvements possibles, en biasant le choix aléatoire par la qualité prédite du mouvement par gnugo (les meilleurs mouvements ont une plus forte probabilité d'être tirés).

Les données dont vous disposez sont brutes. Ce sera à vous de proposer un format adéquat pour utiliser ces données en entrée de votre réseau neuronal. 

_PS_: Je viens de me rendre compte que j'avais rédigé la fin du sujet en anglais (à forcer de mélanger texte et code, je me suis perdu). Pardon.

## Comment sera évalué votre modèle ?

Nous vous fournirons 6h avant la date de rendu un nouveau fichier contenant 1000 nouveaux exemples, qui ne contiendront pas les champs `black_wins`, `white_wins`, `black_points` et `white_points`. Vous devrez laisser, dans votre dépot de projet (votre dépot GIT sous un sous-répertoire ML) un fichier texte nommé `my_predictions.txt` ayant une prédiction par ligne (un simple flottant) qui donnera, dans le même ordre de la liste des exemples les scores que vous prédisez pour chacune des entrées du fichier que nous vous aurons donné. ** Les scores seront donnés sous forme d'un flottant, entre 0 et 1, donnant la probabilité de victoire de noir sur le plateau considéré **. Il faudra laisser, dans votre feuille notebook (voir tout en dessous) la cellule Python qui aura créé ce fichier, pour que l'on puisse éventuellement refaire vos prédictions.

Bien entendu, vous nous rendrez également votre feuille jupyter **sous deux formats**, à la fois le fichier `.ipynb` et le fichier `.html` nous permettant de lire ce que vous avez fait, sans forcément relancer la feuille. Nous prendrons en compte les résultats obtenus sur les prédictions mais aussi le contenu de vos notebooks jupyter.

### Comment sera noté ce TP ?

**Il s'agit d'un TP noté, donc il ne faudra pas y passer trop de temps**. Nous attendons des prédictions correctes mais surtout des choix justifiés dans la feuille. Votre feuille notebook sera le plus important pour la notation (n'hésitez pas à mettre des cellules de texte pour expliquer pourquoi vous avez été amenés à faire certains choix). Ainsi, il serait bien d'avoir, par exemple, les données (graphiques ou autre) qui permettent de comprendre comment vous avez évité l'overfitting.

Le fichier de vos prédiction sera évalué en se basant sur la qualité de vos prédictions. Nous mesurerons par exemple (juste pour vous donner une idée) le nombre d'exemples dont votre prédiction donnera la bonne valeur à 5%, 10%, 20%, 35%, 50% pour estimer sa qualité.


## Mise en route !

Voyons comment lire les données

In [2]:
# Import du fichier d'exemples
from sklearn.model_selection import train_test_split


def get_raw_data_go():
    ''' Returns the set of samples from the local file or download it if it does not exists'''
    import gzip, os.path
    import json

    raw_samples_file = "samples-9x9.json.gz"

    if not os.path.isfile(raw_samples_file):
        print("File", raw_samples_file, "not found, I am downloading it...", end="")
        import urllib.request
        urllib.request.urlretrieve("https://www.labri.fr/perso/lsimon/ia-inge2/samples-9x9.json.gz",
                                   "samples-9x9.json.gz")
        print(" Done")

    with gzip.open("samples-9x9.json.gz") as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data


data = get_raw_data_go()
print("We have", len(data), "examples")

We have 41563 examples


## Compréhension des données de chaque entrée

Voici une description de chaque exemple

In [3]:
def summary_of_example(data, sample_nb):
    ''' Gives you some insights about a sample number'''
    sample = data[sample_nb]
    print("Sample", sample_nb)
    print()
    print("Données brutes en format JSON:", sample)
    print()
    print("The sample was obtained after", sample["depth"], "moves")
    print("The successive moves were", sample["list_of_moves"])
    print("After these moves and all the captures, there was black stones at the following position",
          sample["black_stones"])
    print("After these moves and all the captures, there was white stones at the following position",
          sample["white_stones"])
    print("Number of rollouts (gnugo games played against itself from this position):", sample["rollouts"])
    print("Over these", sample["rollouts"], "games, black won", sample["black_wins"], "times with",
          sample["black_points"], "total points over all this winning games")
    print("Over these", sample["rollouts"], "games, white won", sample["white_wins"], "times with",
          sample["white_points"], "total points over all this winning games")


summary_of_example(data, 10)

Sample 10

Données brutes en format JSON: {'depth': 8, 'list_of_moves': ['C6', 'E7', 'C3', 'C8', 'E5', 'G3', 'B7', 'H7'], 'black_stones': ['B7', 'C6', 'E5', 'C3'], 'white_stones': ['C8', 'E7', 'H7', 'G3'], 'rollouts': 100, 'black_wins': 60, 'black_points': 876.0, 'white_wins': 40, 'white_points': 378.0}

The sample was obtained after 8 moves
The successive moves were ['C6', 'E7', 'C3', 'C8', 'E5', 'G3', 'B7', 'H7']
After these moves and all the captures, there was black stones at the following position ['B7', 'C6', 'E5', 'C3']
After these moves and all the captures, there was white stones at the following position ['C8', 'E7', 'H7', 'G3']
Number of rollouts (gnugo games played against itself from this position): 100
Over these 100 games, black won 60 times with 876.0 total points over all this winning games
Over these 100 games, white won 40 times with 378.0 total points over all this winning games


## Données en entrée et en sortie de votre modèle final

Même si en interne, votre modèle va manipuler des tenseurs en numpy, vous devrez construire une boite noire qui prendra en entrée les coordonnées des pierres noires et blanches et donnera le pourcentage de chance pour noir de gagner depuis cette position. 

Ainsi, pour l'exemple `i` :
- Vous prendez en entree `data[i]["black_stones"]` et `data[i]["white_stones"]` (vous pouvez, si vous le souhaitez, prendre en compte également `list_of_moves` ou tout autre donnée que vous calculerez vous-même (mais qui ne se basera évidemment pas sur les données que vous n'aurez pas lors de l'évaluation).
- Vous devrez prédire simplement `data[i]["black_wins"]/data[i]["rollouts"]`

Encore une fois, **attention** : en interne, il faudra absolument construire vos données formattées en matrices numpy pour faire votre entrainement. On vous demande juste ici d'écrire comment vous faites ces transformations, pour comprendre ce que vous avez décidé de mettre en entrée du réseau.

Voici le modèle de la fonction qui pourra être appelée, au final :


In [4]:
def position_predict(black_stones, white_stones):
    # ... Votre tambouille interne pour placer les pierres comme il faut dans votre structure de données
    # et appeler votre modèle Keras (typiquement avec model.predict())
    prediction = None  # model.predict(...) # A REMPLIR CORRECTEMENT

    return prediction


# Par exemple, nous pourrons appeler votre prédiction ainsi

print("Prediction this sample:")
summary_of_example(data, 10)
print()
prediction = position_predict(data[10]["black_stones"], data[10]["white_stones"])
print("You predicted", prediction, "and the actual target was", data[10]["black_wins"] / data[10]["rollouts"])


# Ainsi, pour le rendu, en admettant que newdata soit le fichier json contenant les nouvelles données que
# l'on vous donnera 24h avant la fin, vous pourrez construire le fichier resultat ainsi

def create_result_file(newdata):
    ''' Exemple de méthode permettant de générer le fichier de resultats demandés. '''
    resultat = [position_predict(d["black_stones"], d["white_stones"]) for d in newdata]
    with open("my_predictions.txt", "w") as f:
        for p in resultat:
            f.write(str(p) + "\n")



Prediction this sample:
Sample 10

Données brutes en format JSON: {'depth': 8, 'list_of_moves': ['C6', 'E7', 'C3', 'C8', 'E5', 'G3', 'B7', 'H7'], 'black_stones': ['B7', 'C6', 'E5', 'C3'], 'white_stones': ['C8', 'E7', 'H7', 'G3'], 'rollouts': 100, 'black_wins': 60, 'black_points': 876.0, 'white_wins': 40, 'white_points': 378.0}

The sample was obtained after 8 moves
The successive moves were ['C6', 'E7', 'C3', 'C8', 'E5', 'G3', 'B7', 'H7']
After these moves and all the captures, there was black stones at the following position ['B7', 'C6', 'E5', 'C3']
After these moves and all the captures, there was white stones at the following position ['C8', 'E7', 'H7', 'G3']
Number of rollouts (gnugo games played against itself from this position): 100
Over these 100 games, black won 60 times with 876.0 total points over all this winning games
Over these 100 games, white won 40 times with 378.0 total points over all this winning games

You predicted None and the actual target was 0.6


In [5]:
% matplotlib inline
import matplotlib.pyplot as plt

plt.title("Relationship between the depth of the board and the chance for black to win")
plt.plot([sample["black_wins"] for sample in data], [sample["depth"] for sample in data], '.')
plt.xlabel("black wins (percentage)")
plt.ylabel("depth of the game")

# Cumulative Distribution function of the chance of black to win
cdf_wins = sorted([sample["black_wins"] for sample in data])
plt.figure()
plt.plot([x / len(cdf_wins) for x in range(len(cdf_wins))], cdf_wins)
plt.title("Cumulative Distribution function of the chance of black to win")
plt.xlabel("\% of the samples with a chance of black to win below y")
plt.ylabel("Chance of black to win")
print("The CDF curve shows that black has more chances to win, globally")

UsageError: Line magic function `%` not found.


# First steps: transform all the data into numpy arrays to feed your neural network

Prérequis : la première cellule du notebook a été exécutée.

Cette étape permet de transformer les données brutes en données exploitables par notre réseau.
Elle repose sur la mise en place d'une stratégie d'élargissement des données en utilisant les symétries et rotations

Après cette étape nous avons environ 500 000 tableaux de taille (2, 9, 9) (un channel pour les pions blancs, un pour les pions noirs)

In [6]:
import numpy as np


def name_to_coord(s):
    assert s != "PASS"
    indexLetters = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'J': 8}

    col = indexLetters[s[0]]
    lin = int(s[1:]) - 1
    return col, lin


# Élargissement des données en utilisant les symétries et rotations
def make_up_down_symetry(elem):
    """
    Fonction permettant de faire une symétrie horizontale
    """
    new_symmetry = []
    for depth in elem:
        new_symmetry.append(np.flipud(depth))
    return np.array(new_symmetry)


def make_left_right_symetry(elem):
    """
    Fonction permettant de faire une symétrie verticale
    """
    new_symmetry = []
    for depth in elem:
        new_symmetry.append(np.fliplr(depth))
    return np.array(new_symmetry)


def make_90_rotation(elem):
    """
    Fonction permettant de faire une rotation à 90°
    """
    new_rotation = []
    for depth in elem:
        new_rotation.append(np.rot90(depth))
    return np.array(new_rotation)


def make_180_rotation(elem):
    """
    Fonction permettant de faire une rotation à 180°
    """
    new_rotation = []
    for depth in elem:
        new_rotation.append(np.rot90(np.rot90(depth)))
    return np.array(new_rotation)


def make_270_rotation(elem):
    """
    Fonction permettant de faire une rotation à 270°
    """
    new_rotation = []
    for depth in elem:
        new_rotation.append(np.rot90(np.rot90(np.rot90(depth))))
    return np.array(new_rotation)


def enlarge_data(data_set, symmetries_rotation_settings, verbose=False):
    """
    Cette fonction permet d'élargir les données à partir d'un set de données déjà existant en utilisant les symétries et
    rotations définies dans les symmetries_rotation_settings
    @param data_set: le set de data initial
    @param symmetries_rotation_settings: les symétries et rotations que l'on veut appliquer
    @return: le set de données élargi
    """

    y_data = np.array([d["black_wins"] / d["rollouts"] for d in data])
    enlarged = []
    y = []
    for e, elem in enumerate(data_set):
        if verbose: print(f"Orginal:{elem.dtype}\n", elem)
        enlarged.append(elem)
        y.append(y_data[e])

        # Élargissement selon les symétries
        if symmetries_rotation_settings["UP_DOWN"]:
            s1 = make_up_down_symetry(elem)
            if verbose: print(f"Symmetry 1{s1.dtype}:\n", s1)
            enlarged.append(s1)
            y.append(y_data[e])
        if symmetries_rotation_settings["LEFT_RIGHT"]:
            s2 = make_left_right_symetry(elem)
            if verbose: print("Symmetry 2:\n", s2)
            enlarged.append(s2)
            y.append(y_data[e])

        # Élargissement selon les rotations
        if symmetries_rotation_settings["deg90"]:
            r1 = make_90_rotation(elem)
            if verbose: print("Rotation 1:\n", r1)
            enlarged.append(r1)
            y.append(y_data[e])
        if symmetries_rotation_settings["deg180"]:
            r2 = make_180_rotation(elem)
            if verbose: print("Rotation 2:\n", r2)
            enlarged.append(r2)
            y.append(y_data[e])
        if symmetries_rotation_settings["deg270"]:
            r3 = make_270_rotation(elem)
            if verbose: print("Rotation 3:\n", r3)
            enlarged.append(r3)
            y.append(y_data[e])

        # Élargissement selon la combinaison des rotations et symétries
        if symmetries_rotation_settings["UP_DOWN"]:
            if symmetries_rotation_settings["deg90"]:
                c1 = make_90_rotation(s1)
                if verbose: print("Combinaison 1:\n", c1)
                enlarged.append(c1)
                y.append(y_data[e])
            if symmetries_rotation_settings["deg180"]:
                c2 = make_180_rotation(s1)
                if verbose: print("Combinaison 2:\n", c2)
                enlarged.append(c2)
                y.append(y_data[e])
            if symmetries_rotation_settings["deg270"]:
                c3 = make_270_rotation(s1)
                if verbose: print("Combinaison 3:\n", c3)
                enlarged.append(c3)
                y.append(y_data[e])
        if symmetries_rotation_settings["LEFT_RIGHT"]:
            if symmetries_rotation_settings["deg90"]:
                c4 = make_90_rotation(s2)
                if verbose: print("Combinaison 4:\n", c4)
                enlarged.append(c4)
                y.append(y_data[e])
            if symmetries_rotation_settings["deg180"]:
                c5 = make_180_rotation(s2)
                if verbose: print("Combinaison 5:\n", c5)
                enlarged.append(c5)
                y.append(y_data[e])
            if symmetries_rotation_settings["deg270"]:
                c6 = make_270_rotation(s2)
                if verbose: print("Combinaison 6:\n", c6)
                enlarged.append(c6)
                y.append(y_data[e])
    return np.array(enlarged), np.array(y)


def fill(black_stones, white_stones):
    """
    Permet de créer notre vecteur de donnée simple à partir des données brutes
    """
    board = np.zeros((2, 9, 9), dtype=int)
    for i in black_stones:
        k, l = name_to_coord(i)
        board[0, l, k] = 1
    for j in white_stones:
        k, l = name_to_coord(j)
        board[1, l, k] = 1
    return board


# Création du vecteur de données simple
x_data = np.array([fill(data[i]["black_stones"], data[i]["white_stones"]) for i in range(0, len(data))])
# Définition des rotations et symétries qui vont être utilisées pour l'élargissement (toutes)
symmetries_rotation_settings = {"UP_DOWN": True, "deg90": True, "deg180": True, "deg270": True, "LEFT_RIGHT": True}
# Élargissement du vecteur de données
x_data, y_data = enlarge_data(x_data, symmetries_rotation_settings)
print("Result : ", x_data.shape, y_data.shape)

Result :  (498756, 2, 9, 9) (498756,)


# Second steps: build your neural network and train it

La construction de ce modèle est grandement inspiré des TD précédent il comporte :
- 3 convolutions 2D séparés par des BatchNormalization
- un aplatissement suivi d'un dropout (10%)
- 3 couches denses avec une activation sur 'relu'
- 1 couche dense avec une activation sigmoid

Le modèle est compilé avec l'optimiseur adam et la fonction de loss utilisée est la mae.

Nous partitionnons aussi les données en deux ensemble (67% de train et 33% de test)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.callbacks import Callback


class History(Callback):
    """
    Cette classe permet de stocker les callback d'un entrainement,
    Elle est emprunté au TD 2 de ML
    """

    def __init__(self):
        super().__init__()
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            if not k in self.history:
                self.history[k] = v


history = [History()]


def create_model():
    """
    Fonction permettant de définir notre modèle
    @return: le modèle créé
    """
    model = Sequential([
        Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(2, 9, 9)),
        BatchNormalization(),
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Flatten(),
        Dropout(0.1),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid'),
    ])

    model.summary()

    model.compile(loss='mae',
                  optimizer='adam',
                  metrics=['mae'])
    return model


EPOCHS = 50
model = create_model()
hist = model.fit(x_data, y_data, batch_size=128, callbacks=history, epochs=EPOCHS, validation_split=0.33)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 2, 9, 16)          1312      
_________________________________________________________________
batch_normalization (BatchNo (None, 2, 9, 16)          64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 9, 32)          4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 9, 32)          128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 9, 64)          18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 2, 9, 64)          256       
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

Epoch 47/50
2611/2611 [==============================] - 38s 15ms/step - loss: 0.0801 - mae: 0.0801 - val_loss: 0.0986 - val_mae: 0.0986
Epoch 48/50
2611/2611 [==============================] - 38s 15ms/step - loss: 0.0789 - mae: 0.0789 - val_loss: 0.0983 - val_mae: 0.0983
Epoch 49/50
2611/2611 [==============================] - 38s 15ms/step - loss: 0.0808 - mae: 0.0808 - val_loss: 0.0994 - val_mae: 0.0994
Epoch 50/50
2611/2611 [==============================] - 38s 15ms/step - loss: 0.0795 - mae: 0.0795 - val_loss: 0.0999 - val_mae: 0.0999


Le modèle est maintenant entrainé nous traçons les courbes de loss et mae pour vérifier les résultats et l'overfitting

In [8]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(nrows=2,
                               ncols=1,
                               figsize=(10, 6),
                               sharex=True,
                               gridspec_kw={'height_ratios': [5, 2]})

ax1.plot(range(EPOCHS), hist.history['mae'], '-o', color='#1f77b4', label='Train')
ax1.plot(range(EPOCHS), hist.history['val_mae'], '-o', color='#ff7f0e', label='Test')
ax1.grid(True)
ax1.set_title("Model MAE")
ax1.set(xlabel="Epoch", ylabel="Mean Absolute Error")
ax1.legend()

ax2.plot(range(EPOCHS), hist.history['loss'], '-o', color='#1f77b4', label='Train')
ax2.plot(range(EPOCHS), hist.history['val_loss'], '-o', color='#ff7f0e', label='Test')
ax2.grid(True)
ax2.set_title("Model loss")
ax2.set(xlabel="Epoch", ylabel="loss")

ax2.legend()
plt.show()

# Last step

Prepare your model to predict the set of new data to predict, you will have only 6 hours to push your predictions.

(may be you would like to express, when guessing the percentage of wins for blacks, that it should reflect the fact that this score should be the same for all the symmetries you considered)...

In [13]:
def get_raw_data_go_evaluate():
    """
    Retourne les données depuis le fichier présent en local ou en téléchargement
    Grandement inspiré de la première fonction du projet
    """
    import gzip, os.path
    import json

    raw_samples_file = "positions-to-evaluate-9x9.json.gz"

    if not os.path.isfile(raw_samples_file):
        print("File", raw_samples_file, "not found, I am downloading it...", end="")
        import urllib.request
        urllib.request.urlretrieve("https://www.labri.fr/perso/lsimon/ia-inge2/positions-to-evaluate-9x9.json.gz",
                                   "samples-9x9.json.gz")
        print(" Done")

    with gzip.open("positions-to-evaluate-9x9.json.gz") as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data


data_evaluate = get_raw_data_go_evaluate()
data_predict_x = np.array([fill(d["black_stones"], d["white_stones"]) for d in data_evaluate])
prediction = model.predict(data_predict_x, batch_size=128, verbose=0, callbacks=history)

with open("my_predictions.txt", "w") as f:
    for p in prediction:
        f.write(str(p[0]) + "\n")
        print(str(p[0])+"\n")

0.60463554

1.0

1.0

0.5799868

0.93985015

1.0

0.58676517

1.0

0.7050457

2.7693686e-06

1.0

0.84494746

0.7196928

3.2021485e-06

0.49866173

0.86675763

0.99902844

0.4373783

1.0

1.0

0.7797617

0.8689059

0.6056515

0.4281854

0.9888104

0.7663246

0.05808142

3.3880377e-19

0.010586679

1.0

0.7047863

0.64542097

0.9998293

0.43868828

0.22856641

0.912946

0.7494887

0.8657061

0.74381924

1.0

1.0

1.0

0.6513249

1.0005882e-14

0.41984308

0.9159231

1.0

1.0

0.7584256

1.0

1.0

0.5028379

0.62716854

0.9999267

0.888953

0.6545896

0.726125

0.9659867

0.7818568

0.7555771

0.7132204

5.7064193e-05

0.99999535

0.5128725

0.74475884

0.87097824

1.0698181e-32

1.0

1.0

0.65944433

0.99995005

0.6103657

0.6772265

0.5551763

0.53007275

0.78048843

0.9999989

1.0

0.49866173

1.0

0.43553928

1.0

1.0

0.0

0.64349806

0.6188469

0.7965698

5.53155e-23

0.71018

1.0

3.312834e-29

0.5932785

0.9999998

0.93539846

0.49866173

2.5428595e-10

1.0

0.87214816

0.7976358